In [ ]:
# Install libraries
!pip -q install autogluon.tabular scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
# Imports
import os, time, numpy as np, pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
SEED = 42
np.random.seed(SEED)

In [ ]:
# Load dataset from the original example repo
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_df = TabularDataset(f'{data_url}train.csv')
test_df  = TabularDataset(f'{data_url}test.csv')
print(train_df.shape, test_df.shape)
train_df.head()

(10000, 19) (5000, 19)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# Target column per the example
label = 'signature'  # classification label with ~18 classes
# Make sure it's treated as categorical
if train_df[label].dtype != 'object' and str(train_df[label].dtype) != 'category':
    train_df[label] = train_df[label].astype('category')

In [ ]:
# Safe split: use stratify only if every class has >= 2 rows
from sklearn.model_selection import train_test_split

cls_counts = train_df[label].value_counts(dropna=False)
if (cls_counts < 2).any():
    print(f"Note: found {(cls_counts < 2).sum()} class(es) with < 2 rows; "
          "using plain random split (no stratify).")
    train_part, dev_part = train_test_split(
        train_df, test_size=0.2, random_state=SEED, shuffle=True
    )
else:
    train_part, dev_part = train_test_split(
        train_df, test_size=0.2, random_state=SEED, stratify=train_df[label]
    )

print(train_part.shape, dev_part.shape)

Note: found 2 class(es) with < 2 rows; using plain random split (no stratify).
(8000, 19) (2000, 19)


In [ ]:
# Train a quick baseline model
save_dir = '/content/ag_tabular_baseline'
predictor = TabularPredictor(label=label, path=save_dir, problem_type='multiclass', eval_metric='accuracy')
predictor.fit(train_data=train_part, tuning_data=dev_part,
              presets='good_quality', time_limit=180,
              num_bag_folds=0, num_stack_levels=0, keep_only_best=True)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.47 GB / 12.67 GB (90.6%)
Disk Space Avail:   186.09 GB / 225.83 GB (82.4%)
Presets specified: ['good_quality']
Using hyperparameters preset: hyperparameters='light'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "/content/ag_tabular_baseline"
Train Data Rows:    8000
Train Data Columns: 18
Tuning Data Rows:    2000
Tuning Data Columns: 18
Label Column:       signature
Problem Type:       multiclass
Preprocessing data ...
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.99825
Train Data Class Count: 9
Using Feature Generators to p

In [ ]:
# Evaluate on held-out dev and on provided test
print('Dev metrics:')
predictor.evaluate(dev_part, silent=True)
print('\nTest metrics:')
predictor.evaluate(test_df, silent=True)


Dev metrics:

Test metrics:


{'accuracy': 0.946,
 'balanced_accuracy': np.float64(0.725941211857342),
 'mcc': np.float64(0.9338337762173625)}

In [ ]:
# Show model leaderboard on test
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_FULL,0.9470,None,accuracy,1.158864,None,5.357897,1.158864,None,5.357897,1,True,3
1,WeightedEnsemble_L2_FULL,0.9460,None,accuracy,11.068730,None,44.335520,0.010829,None,0.214844,2,True,6
2,RandomForestEntr_FULL,0.9406,None,accuracy,0.316048,None,9.572840,0.316048,None,9.572840,1,True,5
3,LightGBMXT_FULL,0.9400,None,accuracy,9.162271,None,10.688059,9.162271,None,10.688059,1,True,2
4,RandomForestGini_FULL,0.9372,None,accuracy,0.326645,None,8.597086,0.326645,None,8.597086,1,True,4
5,NeuralNetFastAI_FULL,0.7412,None,accuracy,0.094074,None,9.904794,0.094074,None,9.904794,1,True,1
